# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/snakemake_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Body"]]
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

433


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['Initial workflow I have a snakefile that can generate some output from '
 'paired-end data. In this snakefile, I have a rule that "installs" the data '
 'given information stored in the config file (get_raw_data). Then I have a '
 'rule that uses that data to generate intermediate files on which the rest of '
 'the workflow depends (run_tophat). Here are the input and output of these '
 'rules (OPJ stands for os.path.join): rule get_raw_data: output: '
 'OPJ(raw_data_dir, "{lib}_1.fastq.gz"), OPJ(raw_data_dir, '
 '"{lib}_2.fastq.gz"), (More details on the implementation of this rule later) '
 'rule run_tophat: input: transcriptome = OPJ(annot_dir, "dmel-all-r5.9.gff"), '
 'fq1 = OPJ(raw_data_dir, "{lib}_1.fastq.gz"), fq2 = OPJ(raw_data_dir, '
 '"{lib}_2.fastq.gz"), output: junctions = OPJ(output_dir, "{lib}", '
 '"junctions.bed"), bam = OPJ(output_dir, "{lib}", "accepted_hits.bam"), And '
 '(simplifying) my main rule would be something like that: rule all: input: '
 'expand(OPJ(outpu

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['initial', 'workflow', 'have', 'snakefile', 'that', 'can', 'generate', 'some', 'output', 'from', 'paired', 'end', 'data', 'in', 'this', 'snakefile', 'have', 'rule', 'that', 'installs', 'the', 'data', 'given', 'information', 'stored', 'in', 'the', 'config', 'file', 'get_raw_data', 'then', 'have', 'rule', 'that', 'uses', 'that', 'data', 'to', 'generate', 'intermediate', 'files', 'on', 'which', 'the', 'rest', 'of', 'the', 'workflow', 'depends', 'run_tophat', 'here', 'are', 'the', 'input', 'and', 'output', 'of', 'these', 'rules', 'opj', 'stands', 'for', 'os', 'path', 'join', 'rule', 'get_raw_data', 'output', 'opj', 'raw_data_dir', 'lib', 'fastq', 'gz', 'opj', 'raw_data_dir', 'lib', 'fastq', 'gz', 'more', 'details', 'on', 'the', 'implementation', 'of', 'this', 'rule', 'later', 'rule', 'run_tophat', 'input', 'transcriptome', 'opj', 'annot_dir', 'dmel', 'all', 'gff', 'fq', 'opj', 'raw_data_dir', 'lib', 'fastq', 'gz', 'fq', 'opj', 'raw_data_dir', 'lib', 'fastq', 'gz', 'output', 'junctions', 

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['initial', 'workflow', 'snakefile', 'generate', 'output', 'datum', 'snakefile', 'rule', 'install', 'datum', 'give', 'information', 'store', 'file', 'get_raw_data', 'rule', 'use', 'datum', 'generate', 'intermediate', 'file', 'rest', 'workflow', 'depend', 'input', 'output', 'rule', 'opj', 'stand', 'os_path', 'join', 'rule', 'output', 'detail', 'implementation', 'rule', 'later', 'rule', 'input', 'transcriptome', 'opj', 'gz', 'output', 'junction', 'opj', 'bam', 'bam', 'simplify', 'main', 'rule', 'something_like', 'rule', 'input', 'expand', 'opj', 'lib', 'extend', 'single_end', 'datum', 'run', 'workflow', 'datum', 'single_end', 'avoid', 'final', 'output', 'different', 'name', 'pattern', 'depend', 'datum', 'single', 'paired_end', 'easily', 'make', 'variant', 'mention', 'rule', 'work', 'single_end', 'datum', 'input', 'output', 'follow', 'rule', 'output', 'rule', 'input', 'transcriptome', 'opj', 'gz', 'output', 'junction', 'opj', 'bam', 'bam', 'provide', 'snakemake', 'enough', 'information',

# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 3), (4, 1), (5, 3), (6, 4), (7, 1), (8, 1), (9, 2), (10, 2), (11, 4), (12, 2), (13, 1), (14, 4), (15, 2), (16, 2), (17, 1), (18, 2), (19, 1), (20, 2), (21, 5), (22, 2), (23, 15), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 4), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 2), (37, 2), (38, 2), (39, 1), (40, 1), (41, 1), (42, 3), (43, 1), (44, 1), (45, 4), (46, 10), (47, 1), (48, 4), (49, 4), (50, 3), (51, 3), (52, 3), (53, 5), (54, 2), (55, 1), (56, 3), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 5), (63, 1), (64, 11), (65, 2), (66, 1), (67, 1), (68, 1), (69, 4), (70, 2), (71, 1), (72, 4), (73, 4), (74, 4), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 4), (82, 1), (83, 1), (84, 4), (85, 1), (86, 1), (87, 1), (88, 6), (89, 1), (90, 11), (91, 3), (92, 3), (93, 2), (94, 1), (95, 1), (96, 2), (97, 1), (98, 2), (99, 10), (100, 3), (101, 2), (102, 1), (103, 1), (104, 1), (105, 2), (106, 4), (107, 25), (108, 2), (109, 1), (1

In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 2),
  ('achieve', 1),
  ('actual', 1),
  ('add', 3),
  ('ambiguity', 1),
  ('ambiguous', 3),
  ('annotation', 4),
  ('apart', 1),
  ('apparently', 1),
  ('associate', 2),
  ('avoid', 2),
  ('bam', 4),
  ('bed', 2),
  ('behaviour', 1),
  ('branch', 4),
  ('case', 2),
  ('choose', 2),
  ('code', 1),
  ('command', 2),
  ('configuration', 1),
  ('contain', 2),
  ('correct', 5),
  ('data', 2),
  ('datum', 15),
  ('def', 1),
  ('depend', 2),
  ('desire', 1),
  ('detail', 1),
  ('determine', 2),
  ('dictionary', 4),
  ('different', 1),
  ('due', 1),
  ('easily', 1),
  ('edit', 1),
  ('else', 2),
  ('end', 1),
  ('enough', 2),
  ('error', 2),
  ('execute', 2),
  ('execution', 1),
  ('exist', 1),
  ('expand', 1),
  ('expect', 3),
  ('extend', 1),
  ('extra', 1),
  ('fastq', 4),
  ('file', 10),
  ('final', 1),
  ('follow', 4),
  ('format', 4),
  ('function', 3),
  ('generate', 3),
  ('get', 3),
  ('get_raw_data', 5),
  ('give', 2),
  ('guide', 1),
  ('gz', 3),
  ('htw', 2),
  ('identi

In [12]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [43]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

  3%|▎         | 16/540 [00:14<07:52,  1.11it/s]

KeyboardInterrupt: 

In [10]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.622738286362022

Coherence Score:  0.41145452501309315


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.184004 -0.087807       1        1  58.594199
0     -0.155487  0.097143       2        1  33.712125
3      0.114049 -0.025145       3        1   3.503349
4      0.110683 -0.000348       4        1   2.140991
1      0.114759  0.016156       5        1   2.049336, topic_info=              Term         Freq        Total Category  logprob  loglift
116      snakemake  1763.000000  1763.000000  Default  30.0000  30.0000
213         sample  1155.000000  1155.000000  Default  29.0000  29.0000
64           input  1357.000000  1357.000000  Default  28.0000  28.0000
318            log   484.000000   484.000000  Default  27.0000  27.0000
151            job   426.000000   426.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
394   finished_job     3.132745    13.287000   Topic5  -5.9167   2.4428
406     steps_done     3.056958    13.620128   Topic5  -5.9412   2.3935
1011         group     4.560002    65.107696   Topic5  -5.5413   1.2289
333        regular     2.516581    11.123714   Topic5  -6.1357   2.4015
1486        export     2.073246     7.380465   Topic5  -6.3295   2.6179

[280 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
916       1  0.027100  activate
916       2  0.948509  activate
1456      1  0.972244    affect
1456      2  0.030383    affect
411       1  0.945597     align
...     ...       ...       ...
4297      2  0.213854      xdot
4297      4  0.427708      xdot
3398      1  0.472997     zipfq
3398      2  0.236498     zipfq
3398      4  0.236498     zipfq

[609 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 5, 2])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.055*"snakemake" + 0.024*"file" + 0.020*"log" + 0.019*"error" + '
  '0.017*"use" + 0.016*"job" + 0.015*"run" + 0.013*"rule" + 0.009*"script" + '
  '0.009*"output"'),
 (1,
  '0.008*"cdirs_user" + 0.008*"pertoldi_legend" + 0.006*"benchmark" + '
  '0.004*"group" + 0.003*"lab_bereket_public" + 0.003*"po" + 0.003*"tier_ver" '
  '+ 0.003*"runtime" + 0.003*"select_job" + 0.003*"sit"'),
 (2,
  '0.042*"rule" + 0.040*"file" + 0.039*"input" + 0.037*"output" + '
  '0.035*"sample" + 0.021*"snakemake" + 0.014*"wildcard" + 0.013*"fastq" + '
  '0.011*"run" + 0.010*"read"'),
 (3,
  '0.015*"field" + 0.008*"image" + 0.008*"transcript_name" + 0.008*"detect" + '
  '0.007*"pattern" + 0.007*"gene_transcript" + 0.005*"perl_pe" + 0.005*"unit" '
  '+ 0.004*"experiment" + 0.004*"gene"'),
 (4,
  '0.006*"antismash" + 0.003*"merged_read" + 0.003*"ppx_s_downsample" + '
  '0.003*"dot" + 0.003*"pear" + 0.002*"fastqc_premerge" + 0.002*"xdot" + '
  '0.002*"plain" + 0.002*"extension" + 0.002*"move"')]


In [12]:
num_topics = 5
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicSnakeMake.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  40510347  Can snakemake avoid ambiguity when two differe...   
1  40936470  Snakemake complains it can't find the file it ...   
2  41121347                               snakemake LSF issues   
3  41465787                 Using regex in snakemake wildcards   
4  41090178                                   Snakemake + tmux   

                                                Body  RatingsSentiCR  \
0  Initial workflow\n\nI have a snakefile that ca...              -1   
1  Consider the following simple snakefile, which...              -1   
2  I have a pipeline which works just fine in the...               1   
3  I'm using regex in snakemake wildcards but I'v...               1   
4  My workflow often includes PBS job submissions...               1   

  RatingsGPT35  RatingsGPTFineTuned  \
0            1                 -1.0   
1           -1                 -1.0   
2           -1                 -1.0   
3           -1